## Use ip_isr Task to make brighter-fatter kernel from a set of high-intensity flats.
Michael Wood-Vasey  (2018-08-21)

Jim Chiang kindly created a set of Phosim and ImSim flats:
"""
The imsim flat pairs are `/global/cscratch1/sd/jchiang8/desc/imsim_tests/bf_flats/lsst_a_*r.fits` at NERSC.
"""
"""
The phosim flat pairs are `/global/cscratch1/sd/jchiang8/desc/calibration_products/bf_flats/lsst_a*.fits`.  There are three pairs in u at 70k, 80k, and 90k e-/pixel, and three pairs in i for the same signal levels.   Note that these are in the imsim `raw` format since the original phosim runs didn't generate the amplifier files, so I can't (easily) make phosim `raw`s.  For this set and for the imsim flat pairs, you would use `lsst.obs.lsstCam.ImsimMapper` for both on ingest.
"""

In [1]:
! mkdir calib_repo
! echo "lsst.obs.lsstCam.LsstCamMapper" > calib_repo/_mapper

mkdir: cannot create directory ‘calib_repo’: File exists


In [2]:
import lsst.obs.base

In [3]:
import lsst.obs.lsstCam

ModuleNotFoundError: No module named 'lsst.obs.lsstCam'

In [ ]:
# Do biases first since constructDark.py expects them.
from lsst.pipe.tasks.ingest import IngestTask

! ingestImages.py calib_repo /global/cscratch1/sd/jchiang8/desc/calibration_products/bias_files/*.fits --mode link

# arg_string = 'calib_repo /global/cscratch1/sd/jchiang8/desc/calibration_products/bias_files/*.fits --mode link'
# args = ' '.split(arg_string)
# IngestTask.parseAndRun(args=args)

In [ ]:
! constructBias.py calib_repo --rerun wmwv/calib --longlog --id visit=3000000 --batch-type none --cores 4

In [ ]:
from lsst.pipe.drivers.constructCalibs import BiasTask
args = ['calib_repo', '--rerun', 'wmwv/calib', '--longlog','--id', 'visit=3000000',
        '--batch-type', 'none', '--cores', '4']
BiasTask.parseAndSubmit(args=args)

In [ ]:
ingestCalibs.py calib_repo --output calib_repo/CALIB --validity 13000 calib_repo/rerun/wmwv/calib/bias/2022-01-01/*.fits

# Darks next, before flats.
! ingestImages.py calib_repo dark_files/*.fits --mode link

! constructDark.py calib_repo --rerun wmwv/calib_dark --longlog --id visit=4000000 --batch-type none --cores 4

! ingestCalibs.py calib_repo --output calib_repo/CALIB --validity 13000 calib_repo/rerun/wmwv/calib_dark/dark/1994-07-19/*.fits

! constructFlat.py calib_repo --rerun wmwv/calib_flat --longlog --id visit=219976 raft=R34 ccd=S02 --batch-type none

In [ ]:
imsim_raw_glob = '/global/cscratch1/sd/jchiang8/desc/imsim_tests/bf_flats/lsst_a_*r.fits'
phosim_raw_glob = '/global/cscratch1/sd/jchiang8/desc/calibration_products/bf_flats/lsst_a*.fits'

mapper_str = 'lsst.obs.lsstCam.ImsimMapper'

Ingest the calibration files.